# Sumo Enviroment

In [3]:
import os
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.logger import configure
from stable_baselines3.common.callbacks import BaseCallback

import matplotlib.pyplot as plt

import traci
import math
from os import path

class SumoEnv(gym.Env):
    def __init__(self):
        self.close()  # tries to close, if not possible, pass

        traci.start(["sumo", "-c", path.abspath("../SUMO/test.sumocfg")])

        ## VARIABLES ##
        self.bus_num = 5
        self.max_steps = 1000
        self.current_step = 0
        bus_stops_num = 4
        bus_speed_max = 50

        ## SUMO VARIABLES ##
        self.bus_stop_positions = [
            [123, 974, 1872, 2764], [123, 827, 1742, 2702, 3592]]
        self.bus_locations = {"-overlap": "123", "-R2": "259", "-R1": "125",
                              "-R0": "267", "-L3": "117", "-L2": "110", "-L1": "123", "-L0": "120"}
        self.bus_ids = ["bus_r_0_0", "bus_r_0_1",
                        "bus_r_0_2", "bus_r_0_3", "bus_r_0_4"]

        # self.route_names = [["-overlap", "-R2", "-R1", "-R0"], ["-overlap", "-L3", "-L2", "-L1", "-L0"]]
        self.route_lengths = [3591, 4697]
        # self.route_junctions={"J1": ["-L0", "-R0", "-overlap"], "J2": ["-R0", "-R1"], "J3": ["-R1", "-R2"], "J4": ["-L3", "-R2", "-overlap"], "J5": ["-L0", "-L1"], "J7": ["-L1", "-L2"], "J8": ["-L2", "-L3"]}

        self.wait_time = 0
        self.previous_speeds_m_s = [0]*self.bus_num
        self.delta_speed = 0.4
        self.min_speed_before_change = 30
        self.action_delta_speed = {
            -1: (1-self.delta_speed), 1: (1+self.delta_speed)}

        ## GYM INITIALIZATIONS ##

        # actions: [b1, b2 (...)] # each action is either -1 = slow down, 0 = keep speed, 1 = speed up
        self.action_space = gym.spaces.Box(low=np.array(
            [np.float32(-1)]*self.bus_num), high=np.array([np.float32(1)]*self.bus_num), shape=(self.bus_num,), dtype=np.float32)

        # states: [avg_wait_time, b1_speed, b1_pos, b2_speed, b2_pos, (...),  bs1_pos, bs2_pos, bs3_pos, bs4_pos]
        wait_max = 100000
        low_obs = np.zeros([1 + 2*self.bus_num + bus_stops_num])
        high_obs = np.array([wait_max] + [bus_speed_max, self.route_lengths[0]]
                            * self.bus_num + [self.route_lengths[0]]*bus_stops_num)
        self.observation_space = gym.spaces.Box(low=low_obs, high=high_obs, shape=(
            1 + 2*self.bus_num + bus_stops_num,), dtype=np.float32)

    # GYM FUNCTIONS
    def reset(self, seed=None, options=None):
        traci.close()
        self.wait_time = 0
        self.current_step = 0
        traci.start(["sumo", "-c", path.abspath("../SUMO/test.sumocfg")])

        # return self.wait_time, {}
        return np.concatenate(([self.wait_time], np.zeros(2 * self.bus_num), self.bus_stop_positions[0])).astype(np.float32)[:15], {}

    def step(self, action):
        # try:
        next_state = self.sumo_step()

        # set action for each bus: -1 = slow down, 0 = keep speed, 1 = speed up
        vehicles_length = len(traci.vehicle.getIDList())

        for i, bus_action in enumerate(action):
            if i >= vehicles_length:
                break
            bus_action = round(bus_action)
            bus_id = self.bus_ids[i]
            bus_distance_driven = traci.vehicle.getDistance(bus_id)

            if np.sign(bus_distance_driven) == -1:
                break  # if bus hasnt driven yet, skip

            bus_route = traci.vehicle.getRouteID(bus_id)
            bus_position = round(bus_distance_driven % (
                self.route_lengths[0] if (bus_route == "r_0") else self.route_lengths[1]), 3)
            nearest_bus_stop_position = self._find_nearest(
                self.bus_stop_positions[0 if bus_route == "r_0" else 1], bus_position)
            bus_speed_m_s = traci.vehicle.getSpeed(bus_id)
            bus_speed_km_t = bus_speed_m_s * 3.6

            interval = [-22, 3]

            new_speed_m_s = 0

            # if bus should keep speed, set speed to previous speed and if previous speed is 0, set to current speed
            if bus_action == 0:
                new_speed_m_s = bus_speed_m_s if (self.previous_speeds_m_s[i] == 0.0) \
                    else self.previous_speeds_m_s[i]
                traci.vehicle.setSpeed(bus_id, new_speed_m_s)

            # change speed if speed > min_speed_before_change and bus is not at a bus stop
            elif (bus_speed_km_t > self.min_speed_before_change and not
                  (bus_position > nearest_bus_stop_position + interval[0] and bus_position < nearest_bus_stop_position + interval[1])):
                new_speed_m_s = self.action_delta_speed[bus_action] * \
                    traci.vehicle.getSpeed(bus_id)
                # smoothly changes to new speed over 1 second
                traci.vehicle.slowDown(bus_id, new_speed_m_s, 1)

            self.previous_speeds_m_s[i] = new_speed_m_s

        # reward are given if the new waiting time is strictly lower, otherwise punished
        reward = 1 if next_state[0] <= self.wait_time else -1

        # set the wait time to the current wait time
        self.wait_time = next_state[0]

        # check if done
        self.current_step += 1
        done = False
        if (self.current_step >= self.max_steps):
            done = True

        truncated = False
        return np.array(next_state, dtype=np.float32), reward, truncated, done, {}

        # except Exception as e:  # if there is an error, close the simulation
        #   print("An error occurred. Closing simulation.")
        #   print("Error: ", e)
        #   traci.close()

    def render(self):
        pass

    def close(self):
        try:
            traci.close()
        except:
            pass

    def seed(self, seed=None):
        pass

    # SUMO FUNCTIONS
    def sumo_step(self):
        # state [avg_wait_time, b1_speed, b1_pos, b2_speed, b2_pos, (...),  bs1_pos, bs2_pos, bs3_pos, bs4_pos]
        new_state = [0] * (1 + 2 * self.bus_num) + self.bus_stop_positions[0]
        personsWaitingTimeList = []
        traci.simulationStep()

        vehicles = traci.vehicle.getIDList()
        persons = traci.person.getIDList()

        # finds the average waiting time
        for i in range(0, len(persons)):
            personWaitingTime = traci.person.getWaitingTime(persons[i])
            personsWaitingTimeList.append(personWaitingTime)

        persons_waiting_num = len(personsWaitingTimeList)
        new_state[0] = round(sum(personsWaitingTimeList) / persons_waiting_num, 3) if persons_waiting_num > 0 or not np.isnan(
            persons_waiting_num) or not np.isnan(personsWaitingTimeList) else 0.0

        # finds bus speed and position
        for j in range(0, len(vehicles)):
            vehicleId = vehicles[j]
            if traci.vehicle.getRouteID(vehicleId) != "r_0":
                continue

            vehicleSpeed = traci.vehicle.getSpeed(vehicleId)*3.6  # m/s to km/h
            vehiclePosition = traci.vehicle.getDistance(vehicleId) % (self.route_lengths[0]
                                                                      if (traci.vehicle.getRouteID(vehicleId) == "r_0") else self.route_lengths[1])
            new_state[1 + 2*j] = round(vehicleSpeed, 2)
            new_state[2 + 2*j] = round(vehiclePosition, 2)
        return new_state

    def _find_nearest(self, array, value):
        idx = np.searchsorted(array, value, side="left")
        if (idx == len(array) and math.fabs(value - (array[0] + array[idx-1])) < math.fabs(value - array[idx-1])):
            return array[0]
        elif idx > 0 and idx == len(array) or math.fabs(value - array[idx-1]) < math.fabs(value - array[idx]):
            return array[idx-1]
        else:
            return array[idx]

#### Manual Testing

In [2]:
import random 
# Test the environment
env = SumoEnv()

# Reset the environment to get the initial state
state, info = env.reset()

# Loop for multiple simulation steps
for _ in range(1000):
  # Take an action (array of integers representing speed changes for each bus)
  # Example action (keep speed, slow down, speed up, ...)
  #action = [1, 0, 2, 1, 0]
  action = [random.randint(-1, 1) for i in range(5)]

  # Perform a step in the environment
  next_state, reward, done, info, truncated = env.step(action)


  # Process the reward and next state
  np.set_printoptions(suppress=True)
  print(f"Step: {_}, Wait Time: {next_state[0]}, Reward: {reward}, Next State: {next_state}, Done: {done}, Info: {info} \n")

  # Check if the simulation is done
  if done:
    break

# Close the environment
env.close()

 Retrying in 1 seconds


Step #0.00 (0ms ?*RT. ?UPS, TraCI: 2ms, vehicles TOT 0 ACT 0 BUF 0)                      
 Retrying in 1 seconds


Step: 0, Wait Time: 0.0, Reward: 1, Next State: [   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  123.
  974. 1872. 2764.], Done: False, Info: False 

Step: 1, Wait Time: 0.5, Reward: -1, Next State: [   0.5     0.      0.      1.1     0.31    0.      0.      0.96    0.27
    0.      0.    123.    974.   1872.   2764.  ], Done: False, Info: False 

Step: 2, Wait Time: 0.6000000238418579, Reward: -1, Next State: [   0.6     0.      0.      1.18    0.63    0.      0.      0.96    0.53
    0.      0.    123.    974.   1872.   2764.  ], Done: False, Info: False 

Step: 3, Wait Time: 1.600000023841858, Reward: -1, Next State: [   1.6     0.      0.      1.15    0.95    0.      0.      0.96    0.8
    0.      0.    123.    974.   1872.   2764.  ], Done: False, Info: False 

Step: 4, Wait Time: 1.7139999866485596, Reward: -1, Next State: [   1.714    0.       0.       0.15     1.       0.       0.       0.68
    0.99     0.       0.     123.     974.    1872.    2764.   ], D

Step: 316, Wait Time: 113.4260025024414, Reward: 1, Next State: [ 113.426    0.       0.       0.       1.       0.       0.       0.
  646.56    20.59   731.29   123.     974.    1872.    2764.   ], Done: False, Info: False 

Step: 317, Wait Time: 112.36599731445312, Reward: 1, Next State: [ 112.366    0.       0.       0.       1.       0.       0.       0.
  646.56    20.59   737.01   123.     974.    1872.    2764.   ], Done: False, Info: False 

Step: 318, Wait Time: 111.37699890136719, Reward: 1, Next State: [ 111.377    0.       0.       0.       1.       0.       0.       0.
  646.56    20.59   742.73   123.     974.    1872.    2764.   ], Done: False, Info: False 

Step: 319, Wait Time: 109.91799926757812, Reward: 1, Next State: [ 109.918    0.       0.       0.       1.       0.       0.       0.
  646.56    20.59   748.45   123.     974.    1872.    2764.   ], Done: False, Info: False 

Step: 320, Wait Time: 109.45700073242188, Reward: 1, Next State: [ 109.457    0.       0.

Step: 388, Wait Time: 103.78700256347656, Reward: -1, Next State: [ 103.787    0.       0.       0.       1.       0.     676.16     0.
  646.56    12.96  1035.71   123.     974.    1872.    2764.   ], Done: False, Info: False 

Step: 389, Wait Time: 104.47599792480469, Reward: -1, Next State: [ 104.476    0.       0.       0.       1.       0.     676.16     0.
  646.56    12.96  1039.31   123.     974.    1872.    2764.   ], Done: False, Info: False 

Step: 390, Wait Time: 104.23799896240234, Reward: 1, Next State: [ 104.238    0.       0.       0.       1.       0.     676.16     0.
  646.56    12.96  1042.91   123.     974.    1872.    2764.   ], Done: False, Info: False 

Step: 391, Wait Time: 104.46900177001953, Reward: -1, Next State: [ 104.469    0.       0.       0.       1.       0.     676.16     0.
  646.56    12.96  1046.51   123.     974.    1872.    2764.   ], Done: False, Info: False 

Step: 392, Wait Time: 106.56400299072266, Reward: -1, Next State: [ 106.564    0.    

Step: 485, Wait Time: 120.1760025024414, Reward: -1, Next State: [ 120.176    0.     735.36     0.     705.76     0.     676.16     0.
  646.56    12.96  1384.91   123.     974.    1872.    2764.   ], Done: False, Info: False 

Step: 486, Wait Time: 120.37899780273438, Reward: -1, Next State: [ 120.379    0.     735.36     0.     705.76     0.     676.16     0.
  646.56    12.96  1388.51   123.     974.    1872.    2764.   ], Done: False, Info: False 

Step: 487, Wait Time: 121.01399993896484, Reward: -1, Next State: [ 121.014    0.     735.36     0.     705.76     0.     676.16     0.
  646.56    12.96  1392.11   123.     974.    1872.    2764.   ], Done: False, Info: False 

Step: 488, Wait Time: 121.21700286865234, Reward: -1, Next State: [ 121.217    0.     735.36     0.     705.76     0.     676.16     0.
  646.56    12.96  1395.71   123.     974.    1872.    2764.   ], Done: False, Info: False 

Step: 489, Wait Time: 121.4219970703125, Reward: -1, Next State: [ 121.422    0.     

Step: 789, Wait Time: 162.42599487304688, Reward: -1, Next State: [ 162.426    0.    1765.       0.    1754.5      0.    1715.5     11.09
 1748.74    11.09  2376.43   123.     974.    1872.    2764.   ], Done: False, Info: False 

Step: 790, Wait Time: 162.48300170898438, Reward: -1, Next State: [ 162.483    0.    1765.       0.    1754.5      0.    1715.5     11.09
 1751.82    11.09  2379.51   123.     974.    1872.    2764.   ], Done: False, Info: False 

Step: 791, Wait Time: 163.10499572753906, Reward: -1, Next State: [ 163.105    0.    1765.       0.    1754.5      0.    1715.5     11.09
 1754.9     11.09  2382.59   123.     974.    1872.    2764.   ], Done: False, Info: False 

Step: 792, Wait Time: 163.26300048828125, Reward: -1, Next State: [ 163.263    0.    1765.       0.    1754.5      0.    1715.5     11.09
 1757.98    11.09  2385.67   123.     974.    1872.    2764.   ], Done: False, Info: False 

Step: 793, Wait Time: 163.4239959716797, Reward: -1, Next State: [ 163.424  

## Stable baseline 3

### Checking Sumo Enviroment
Check that an environment follows Gym API. 
It also optionally check that the environment is compatible with Stable-Baselines.

In [23]:
# Check that an environment follows Gym API
# It also optionally check that the environment is compatible with Stable-Baselines.
from stable_baselines3.common.env_checker import check_env

env = SumoEnv()
check_env(env, warn=True, skip_render_check=True)

 Retrying in 1 seconds


Step #0.00 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 0 ACT 0 BUF 0)                      
 Retrying in 1 seconds


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/stable_baselines3/common/env_checker.py:453: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(


Step #0.00 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 0 ACT 0 BUF 0)                      
 Retrying in 1 seconds


Step #1.00 (1ms ~= 1000.00*RT, ~1000.00UPS, TraCI: 31ms, vehicles TOT 1 ACT 1 BUF 1)      
 Retrying in 1 seconds


#### Unit Testing

In [18]:
import unittest

class TestSumoEnv(unittest.TestCase):
  def setUp(self):  # setup the variables here before every test
    self.env = SumoEnv()

  def tearDown(self):  # after each test
    self.env.close()

  @classmethod
  def tearDownClass(cls):  # after all tests are done
    pass

  def test_initializations(self):
    action = [1, 1, 1, 1, 1]
    state, reward, done, info, truncated = self.env.step(action)


  
    self.assertEqual(len(state), 15)
    self.assertTrue((state == (np.array([0.0, 0.0, 0.0, 0, 0, 0, 0,
                                        0, 0, 0, 0, 123, 974, 1872, 2764], dtype=np.float32))).all())
    
    # reward is 1 if new wait time is equal or lower
    self.assertEqual(reward, 1)
    self.assertFalse(done)

  def test_negative_reward(self):
    action = [2, 2, 2, 2, 2]
    self.env.step(action)
    _, reward, *_ = self.env.step(action)
    # reward is -1, since pedestrians have now spawned and waiting time has increased
    self.assertEqual(reward, -1)

  def test_find_nearest_function(self):
    self.assertEqual(self.env._find_nearest(
        [123, 827, 1742, 2702, 3592], 3800), 123)  # goes into first if statement
    self.assertEqual(self.env._find_nearest(
        [123, 827, 1742, 2702, 3592], 10000), 123)  # goes into first if statement
    # goes into second if statement
    self.assertEqual(self.env._find_nearest(
        [123, 827, 1742, 2702, 3592], 950), 827)
    self.assertEqual(self.env._find_nearest(
        [123, 827, 1742, 2702, 3592], 1800), 1742)  # goes into second if statement
    self.assertEqual(self.env._find_nearest(
        [123, 827, 1742, 2702, 3592], 3592), 3592)  # goes into else if statement
    self.assertEqual(self.env._find_nearest(
        [123, 827, 1742, 2702, 3592], 123), 123)  # goes into else if statement

  def test_do_15_steps_keep_speed(self):
    action = [1, 1, 1, 1, 1]  # keep speed

    for _ in range(10):
      self.env.step([2, 2, 2, 2, 2])

    state, *_ = self.env.step(action)
    keep_speed_km_t = state[1]
    for _ in range(4):
      state, *_ = self.env.step(action)
      self.assertEqual(state[1], keep_speed_km_t)

  def test_do_15_steps_speed_up(self):
    action = [2, 2, 2, 2, 2]  # speed up

    for _ in range(10):
      self.env.step(action)

    previous_speed = 0
    for _ in range(5):
      state, *_ = self.env.step(action)
      self.assertGreaterEqual(state[1], previous_speed)
      previous_speed = state[1]

  def test_do_15_steps_slow_down(self):
    action = [0, 0, 0, 0, 0]  # slow down

    # assumes min speed before changing speed is 30km/t
    self.assertEqual(self.env.min_speed_before_change, 30)
    for _ in range(11):  # reaches 30km/t at 11th step
      self.env.step(action)

    for _ in range(4):  # should keep speed lesser than 30km/t
      state, *_ = self.env.step(action)
      self.assertLessEqual(state[1], self.env.min_speed_before_change)

  def test_all_buses_has_driven_after_202_steps(self):
    action = [2, 2, 2, 2, 2]
    for _ in range(201):
      self.env.step(action)
    state, *_ = self.env.step(action)

    # check that all buses have driven a distance greater than 0
    # edge case: bus position is the route length, and pos % route length is 0 (handled: this busses cannot drive far enough)
    indicies_route_position = [2, 4, 6, 8]
    for i in indicies_route_position:
      self.assertGreater(state[i], 0)


unittest.main(argv=[''], exit=False)

 Retrying in 1 seconds


.

Step #202.00 (1ms ~= 1000.00*RT, ~9000.00UPS, TraCI: 41ms, vehicles TOT 9 ACT 9 BUF 1)    
 Retrying in 1 seconds


.

Step #15.00 (0ms ?*RT. ?UPS, TraCI: 27ms, vehicles TOT 2 ACT 2 BUF 0)                     
 Retrying in 1 seconds


...

Step #0.00 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 0 ACT 0 BUF 0)                      
 Retrying in 1 seconds


.

Step #1.00 (1ms ~= 1000.00*RT, ~1000.00UPS, TraCI: 34ms, vehicles TOT 1 ACT 1 BUF 1)      
 Retrying in 1 seconds


.
----------------------------------------------------------------------
Ran 7 tests in 29.048s

OK


Step #2.00 (0ms ?*RT. ?UPS, TraCI: 22ms, vehicles TOT 1 ACT 1 BUF 1)                      


# Stable baseline 3 PPO Implementation